In [1]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True

if OWN_INSTALL:
    print("Own install")
    sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',
                     '../../../..',  '../../../../3rdparty',
                     '../../..',     '../../../3rdparty',
                     '../..',        '../../3rdparty',
                     '..',           '../3rdparty' ]

else: # In colab
    ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
    sys.path.append('./Jove')
    sys.path.append('./Jove/jove')

# -- common imports --
from jove.PcpJupyterNew import *
from jove.Def_TM     import *
from jove.Def_md2mc  import *
from jove.DotBashers import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Cloning into 'Jove'...
remote: Enumerating objects: 6770, done.
remote: Counting objects: 100% (2136/2136), done.
remote: Compressing objects: 100% (810/810), done.
remote: Total 6770 (delta 1306), reused 2056 (delta 1243), pack-reused 4634 (from 1)
Receiving objects: 100% (6770/6770), 11.45 MiB | 10.43 MiB/s, done.
Resolving deltas: 100% (4311/4311), done.
You may use any of these help commands:
help(step_tm)
help(run_tm)
help(explore_tm)

You may use any of these help commands:
help(md2mc)
.. and if you want to dig more, then ..
help(default_line_attr)
help(length_ok_input_items)
help(union_line_attr_list_fld)
help(extend_rsltdict)
help(form_delta)
help(get_machine_components)

You may use any of these help commands:
help(ResetStNum)
help(NxtStateStr)




# NOTE THAT A HELPFUL VIDEO IS PRESENTED ON CANVAS for the basics of Asg6

# It is also the first video in the module REMAINING VIDEOS (NOV, DEC) 2022

#===
# *INSTRUCTIONS ON WHAT TO UPLOAD*

1) Fully executed ipynb notebooks

2) One PDF screenshot (or enough screenshots) to cover the screen output after you execute the tests

3) The animation of ONE of your DTM tests and animation of ONE of your NDTM tests (choose any run)

4) Try generating a PDF print of the whole screen, and if it succeeds, upload that also

5) Please name your PDFs as asg07-file1.pdf asg07-file2.pdf etc so that we can process these PDFs in some sensible way

#===

# Question 1 (50 points):

## The TM design is *25 points* for a well-documented TM

## Passing the given tests fetches 3 points each - total *15 points*

## Each test you add (5 requested) that passes fetches 2 point each - total *10 points*

## Strive to make three of your tests involve input strings of length 10 or more (to avoid having entirely short tests)

- Design a deterministic Turing machine (DTM) over the input alphabet Σ = {a, b, #} for the language of all strings of the form w#x where w, x ∈ {a, b}∗ and w is a substring of x. See the definition of substring here: https://en.wikipedia.org/wiki/Substring

- You must design a deterministic Turing machine which absolutely has no non-determinism anywhere in its state transitions. A deterministic algorithm here requires searching for all w beginnings within x. A correct design will have a deterministic polynomial runtime.

- More specifically, you will start the search at the beginning of x, fail, move one position forward on x, start the search again, etc., till you cannot find a position within x to start the search on

- You don't need to do any error-checks of user input. Assume that the user never gave anything outside of the above $\Sigma$ in their input.

- Add 5 more tests of your own.

In [2]:
Asg6DTM = md2mc('''TM
!! PLAN:
!! Mark an X for a 0 or a Y for a 1
!! Read to past the # and any subsequent F's X's or Y's
!! If the next 0 or 1 doesn't match what we just set, this substring is INVALID.
!!    Move back, setting Xs to 0s and Ys to 1, until we get to midpoint #
!!    Move forward and past any F's, and set the first 0 or 1 to an F to mark that that is NOT the start of the substring.
!!    Move back to the start of the string, resetting Xs and Ys, and restart.
!! If the 0 or 1 DOES match, set it to X or Y, the substring is valid so far.
!!    Loop back to the last X or Y before the midpoint.
!!    Then loop forward once, and repeat with the next char of the substring.
!! If we read a # in our reading loop it means the complete substring is valid, so move to an accept state.

!! Reading loop

IqR0 : #;#,S -> Fq0                  !! Accept if we've reached our midpoint. Also accepts empty substring (.#w) as that is contained in all strings w

IqR0 : a;X,R -> qX0                  !! Run through the Validate X branch
IqR0 : b;Y,R -> qY0                  !! Run through the Validate Y branch

!! Validate X branch - reading through to make sure there is an a next

qX0 : a;a,R | b;b,R -> qX0           !! Read through a's and b's to get to the midpoint
qX0 : #;#,R -> qX1

qX1 : F;F,R | X;X,R | Y;Y,R -> qX1   !! Read through F's, X's, and Y's to get to the character we actually check

qX1 : a;X,L -> qV0                   !! The character was valid so we can mark it and start looping back on the Valid branch
qX1 : b;b,L -> qI0                   !! The character was NOT valid so we must loop back on the Invalid branch and reset.

!! Validate Y branch - reading through to make sure there is a b next
!! Very similar to X

qY0 : a;a,R | b;b,R -> qY0           !! Read through a's and b's to get to the midpoint
qY0 : #;#,R -> qY1

qY1 : F;F,R | X;X,R | Y;Y,R -> qY1   !! Read through F's, X's, and Y's to get to the character we actually check

qY1 : a;a,L -> qI0                   !! The character was NOT valid so we must loop back on the Invalid branch and reset.
qY1 : b;Y,L -> qV0                   !! The character was valid so we can mark it and start looping back on the Valid branch

!! Valid branch - character matched, so we loop back until the next character to check and re-enter the read loop.

qV0 : X;X,L | Y;Y,L | F;F,L -> qV0   !! Read back to midpoint
qV0 : #;#,L -> qV1

qV1 : a;a,L | b;b,L -> qV1           !! Read back all a's and b's (unprocessed)
qV1 : X;X,R | Y;Y,R | .;.,R -> IqR0  !! Begin the read loop from the next unprocessed character

!! Invalid branch - character did NOT match, loop back resetting everything but adding an F to the start of this substring

qI0 : X;a,L | Y;b,L | F;F,L -> qI0   !! Read back to midpoint, resetting X's and Y's
qI0 : #;#,R -> qI1

qI1 : F;F,R -> qI1                   !! Read past any characters that were previously marked as failures
qI1 : a;F,L | b;F,L -> qI2           !! Mark the first character we see as a failure and return left

qI2 : F;F,L -> qI2                   !! Read back to midpoint
qI2 : #;#,L -> qI3

qI3 : X;a,L | Y;b,L -> qI3           !! Read back to the start of the string, resetting X's and Y's
qI3 : a;a,L | b;b,L -> qI3           !! Also read back A's and B's which are unprocessed.
qI3 : .;.,R -> IqR0                  !! At the start of the string, re-enter the read loop.

''')

Generating LALR tables


### Test Cases: The "fuel" argument of 50 may be increased as needed (start with 50 and that is good if all tests finish)

In [3]:
help(explore_tm)

Help on function explore_tm in module jove.Def_TM:

explore_tm(T, tape, nsteps)
    A handy routine to print the result of run_tm plus making 
    future extensions to explore run-results.



In [4]:
explore_tm(Asg6DTM, "aba#aba",50)

Detailing the halted configs now.
Accepted at  ('Fq0', 3, 'XYX#XYX', 22)
 via .. 
 ->('IqR0', 0, 'aba#aba', 50)
 ->('qX0', 1, 'Xba#aba', 49)
 ->('qX0', 2, 'Xba#aba', 48)
 ->('qX0', 3, 'Xba#aba', 47)
 ->('qX1', 4, 'Xba#aba', 46)
 ->('qV0', 3, 'Xba#Xba', 45)
 ->('qV1', 2, 'Xba#Xba', 44)
 ->('qV1', 1, 'Xba#Xba', 43)
 ->('qV1', 0, 'Xba#Xba', 42)
 ->('IqR0', 1, 'Xba#Xba', 41)
 ->('qY0', 2, 'XYa#Xba', 40)
 ->('qY0', 3, 'XYa#Xba', 39)
 ->('qY1', 4, 'XYa#Xba', 38)
 ->('qY1', 5, 'XYa#Xba', 37)
 ->('qV0', 4, 'XYa#XYa', 36)
 ->('qV0', 3, 'XYa#XYa', 35)
 ->('qV1', 2, 'XYa#XYa', 34)
 ->('qV1', 1, 'XYa#XYa', 33)
 ->('IqR0', 2, 'XYa#XYa', 32)
 ->('qX0', 3, 'XYX#XYa', 31)
 ->('qX1', 4, 'XYX#XYa', 30)
 ->('qX1', 5, 'XYX#XYa', 29)
 ->('qX1', 6, 'XYX#XYa', 28)
 ->('qV0', 5, 'XYX#XYX', 27)
 ->('qV0', 4, 'XYX#XYX', 26)
 ->('qV0', 3, 'XYX#XYX', 25)
 ->('qV1', 2, 'XYX#XYX', 24)
 ->('IqR0', 3, 'XYX#XYX', 23)
 ->('Fq0', 3, 'XYX#XYX', 22)


In [5]:
# assuming this is supposed to be rejected
explore_tm(Asg6DTM, "aba#ab",50)

Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('qX1', 6, 'XYX#XY', 28)
 via .. 
 ->('IqR0', 0, 'aba#ab', 50)
 ->('qX0', 1, 'Xba#ab', 49)
 ->('qX0', 2, 'Xba#ab', 48)
 ->('qX0', 3, 'Xba#ab', 47)
 ->('qX1', 4, 'Xba#ab', 46)
 ->('qV0', 3, 'Xba#Xb', 45)
 ->('qV1', 2, 'Xba#Xb', 44)
 ->('qV1', 1, 'Xba#Xb', 43)
 ->('qV1', 0, 'Xba#Xb', 42)
 ->('IqR0', 1, 'Xba#Xb', 41)
 ->('qY0', 2, 'XYa#Xb', 40)
 ->('qY0', 3, 'XYa#Xb', 39)
 ->('qY1', 4, 'XYa#Xb', 38)
 ->('qY1', 5, 'XYa#Xb', 37)
 ->('qV0', 4, 'XYa#XY', 36)
 ->('qV0', 3, 'XYa#XY', 35)
 ->('qV1', 2, 'XYa#XY', 34)
 ->('qV1', 1, 'XYa#XY', 33)
 ->('IqR0', 2, 'XYa#XY', 32)
 ->('qX0', 3, 'XYX#XY', 31)
 ->('qX1', 4, 'XYX#XY', 30)
 ->('qX1', 5, 'XYX#XY', 29)
 ->('qX1', 6, 'XYX#XY', 28)


In [6]:
explore_tm(Asg6DTM, "aba#baba",50)

Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Accepted at  ('Fq0', 11, '........XYX#FXYX', 4)
 via .. 
 ->('IqR0', 0, 'aba#baba', 50)
 ->('qX0', 1, 'Xba#baba', 49)
 ->('qX0', 2, 'Xba#baba', 48)
 ->('qX0', 3, 'Xba#baba', 47)
 ->('qX1', 4, 'Xba#baba', 46)
 ->('qI0', 3, 'Xba#baba', 45)
 ->('qI1', 4, 'Xba#baba', 44)
 ->('qI2', 3, 'Xba#Faba', 43)
 ->('qI3', 2, 'Xba#Faba', 42)
 ->('qI3', 1, 'Xba#Faba', 41)
 ->('qI3', 0, 'Xba#Faba', 40)
 ->('qI3', 7, '........aba#Faba', 39)
 ->('IqR0', 8, '........aba#Faba', 38)
 ->('qX0', 9, '........Xba#Faba', 37)
 ->('qX0', 10, '........Xba#Faba', 36)
 ->('qX0', 11, '........Xba#Faba', 35)
 ->('qX1', 12, '........Xba#Faba', 34)
 ->('qX1', 13, '........Xba#Faba', 33)
 ->('qV0', 12, '........Xba#FXba', 32)
 ->('qV0', 11, '........Xba#FXba', 31)
 ->('qV1', 10, '........Xba#FXba', 30)
 ->('qV1', 9, '........Xba#FXba', 29)
 ->('qV1', 8, '........Xba#FXba', 28)
 ->('IqR0', 9, '........Xba#FXba', 27)
 ->('qY0', 10, '........XYa#FXba', 2

In [7]:
# assuming this is also meant to be rejected
explore_tm(Asg6DTM, "aba#babba",150)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('qY1', 17, '........XYa#FFFFX', 29)
 via .. 
 ->('IqR0', 0, 'aba#babba', 150)
 ->('qX0', 1, 'Xba#babba', 149)
 ->('qX0', 2, 'Xba#babba', 148)
 ->('qX0', 3, 'Xba#babba', 147)
 ->('qX1', 4, 'Xba#babba', 146)
 ->('qI0', 3, 'Xba#babba', 145)
 ->('qI1', 4, 'Xba#babba', 144)
 ->('qI2', 3, 'Xba#Fabba', 143)
 ->('qI3', 2, 'Xba#Fabba', 142)
 ->('qI3', 1, 'Xba#Fabba', 141)
 ->('qI3', 0, 'Xba#Fabba', 140)
 ->('qI3', 7, '........aba#Fabba', 139)
 ->('IqR0', 8, '........aba#Fabba', 138)
 ->('qX0', 9, '........Xba#Fabba', 137)
 ->('qX0', 10, '........Xba#Fabba', 136)
 ->('qX0', 11, '........Xba#Fabba', 135)
 ->('qX1', 12, '........Xba#Fabba', 134)
 ->('qX1', 13, '........Xba#Fabba', 133)
 ->('qV0', 12, '........Xba#FXbba', 132)
 ->('qV0', 11, '........Xba#FXbba', 131)
 ->('qV1', 10, '........Xba#FXbba', 130)
 ->('qV1', 9, '........Xba#FXbba', 129)
 ->('qV1', 8, '........Xba#

In [8]:
explore_tm(Asg6DTM, "bba#bbabba",50)

Detailing the halted configs now.
Accepted at  ('Fq0', 3, 'YYX#YYXbba', 22)
 via .. 
 ->('IqR0', 0, 'bba#bbabba', 50)
 ->('qY0', 1, 'Yba#bbabba', 49)
 ->('qY0', 2, 'Yba#bbabba', 48)
 ->('qY0', 3, 'Yba#bbabba', 47)
 ->('qY1', 4, 'Yba#bbabba', 46)
 ->('qV0', 3, 'Yba#Ybabba', 45)
 ->('qV1', 2, 'Yba#Ybabba', 44)
 ->('qV1', 1, 'Yba#Ybabba', 43)
 ->('qV1', 0, 'Yba#Ybabba', 42)
 ->('IqR0', 1, 'Yba#Ybabba', 41)
 ->('qY0', 2, 'YYa#Ybabba', 40)
 ->('qY0', 3, 'YYa#Ybabba', 39)
 ->('qY1', 4, 'YYa#Ybabba', 38)
 ->('qY1', 5, 'YYa#Ybabba', 37)
 ->('qV0', 4, 'YYa#YYabba', 36)
 ->('qV0', 3, 'YYa#YYabba', 35)
 ->('qV1', 2, 'YYa#YYabba', 34)
 ->('qV1', 1, 'YYa#YYabba', 33)
 ->('IqR0', 2, 'YYa#YYabba', 32)
 ->('qX0', 3, 'YYX#YYabba', 31)
 ->('qX1', 4, 'YYX#YYabba', 30)
 ->('qX1', 5, 'YYX#YYabba', 29)
 ->('qX1', 6, 'YYX#YYabba', 28)
 ->('qV0', 5, 'YYX#YYXbba', 27)
 ->('qV0', 4, 'YYX#YYXbba', 26)
 ->('qV0', 3, 'YYX#YYXbba', 25)
 ->('qV1', 2, 'YYX#YYXbba', 24)
 ->('IqR0', 3, 'YYX#YYXbba', 23)
 ->('Fq0', 3, '

### 5 Tests You Add:

In [9]:
# Test 1
explore_tm(Asg6DTM, "abb#ababb",100)

Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Accepted at  ('Fq0', 11, '........XYY#FFXYY', 12)
 via .. 
 ->('IqR0', 0, 'abb#ababb', 100)
 ->('qX0', 1, 'Xbb#ababb', 99)
 ->('qX0', 2, 'Xbb#ababb', 98)
 ->('qX0', 3, 'Xbb#ababb', 97)
 ->('qX1', 4, 'Xbb#ababb', 96)
 ->('qV0', 3, 'Xbb#Xbabb', 95)
 ->('qV1', 2, 'Xbb#Xbabb', 94)
 ->('qV1', 1, 'Xbb#Xbabb', 93)
 ->('qV1', 0, 'Xbb#Xbabb', 92)
 ->('IqR0', 1, 'Xbb#Xbabb', 91)
 ->('qY0', 2, 'XYb#Xbabb', 90)
 ->('qY0', 3, 'XYb#Xbabb', 89)
 ->('qY1', 4, 'XYb#Xbabb', 88)
 ->('qY1', 5, 'XYb#Xbabb', 87)
 ->('qV0', 4, 'XYb#XYabb', 86)
 ->('qV0', 3, 'XYb#XYabb', 85)
 ->('qV1', 2, 'XYb#XYabb', 84)
 ->('qV1', 1, 'XYb#XYabb', 83)
 ->('IqR0', 2, 'XYb#XYabb', 82)
 ->('qY0', 3, 'XYY#XYabb', 81)
 ->('qY1', 4, 'XYY#XYabb', 80)
 ->('qY1', 5, 'XYY#XYabb', 79)
 ->('qY1', 6, 'XYY#XYabb', 78)
 ->('qI0', 5, 'XYY#XYabb', 77)
 ->('qI0', 4, 'XYY#Xbabb', 76)
 ->('qI0', 3, 'XYY#ababb', 75)
 ->('qI1', 4, 'XYY#ababb', 74)
 ->('qI2', 3, 'XYY#Fbabb', 

In [10]:
# Test 2
explore_tm(Asg6DTM, "ab#bbbbbab",150)

Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Accepted at  ('Fq0', 10, '........XY#FFFFFXY', 25)
 via .. 
 ->('IqR0', 0, 'ab#bbbbbab', 150)
 ->('qX0', 1, 'Xb#bbbbbab', 149)
 ->('qX0', 2, 'Xb#bbbbbab', 148)
 ->('qX1', 3, 'Xb#bbbbbab', 147)
 ->('qI0', 2, 'Xb#bbbbbab', 146)
 ->('qI1', 3, 'Xb#bbbbbab', 145)
 ->('qI2', 2, 'Xb#Fbbbbab', 144)
 ->('qI3', 1, 'Xb#Fbbbbab', 143)
 ->('qI3', 0, 'Xb#Fbbbbab', 142)
 ->('qI3', 7, '........ab#Fbbbbab', 141)
 ->('IqR0', 8, '........ab#Fbbbbab', 140)
 ->('qX0', 9, '........Xb#Fbbbbab', 139)
 ->('qX0', 10, '........Xb#Fbbbbab', 138)
 ->('qX1', 11, '........Xb#Fbbbbab', 137)
 ->('qX1', 12, '........Xb#Fbbbbab', 136)
 ->('qI0', 11, '........Xb#Fbbbbab', 135)
 ->('qI0', 10, '........Xb#Fbbbbab', 134)
 ->('qI1', 11, '........Xb#Fbbbbab', 133)
 ->('qI1', 12, '........Xb#Fbbbbab', 132)
 ->('qI2', 11, '........Xb#FFbbbab', 131)
 ->('qI2', 10, '........Xb#FFbbbab', 130)
 ->('qI3', 9, '........Xb#FFbbbab', 129)
 ->('qI3', 8, '........Xb#

In [11]:
# Test 3
explore_tm(Asg6DTM, "ba#bbbbbab",150)

Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Accepted at  ('Fq0', 10, '........YX#FFFFYXb', 11)
 via .. 
 ->('IqR0', 0, 'ba#bbbbbab', 150)
 ->('qY0', 1, 'Ya#bbbbbab', 149)
 ->('qY0', 2, 'Ya#bbbbbab', 148)
 ->('qY1', 3, 'Ya#bbbbbab', 147)
 ->('qV0', 2, 'Ya#Ybbbbab', 146)
 ->('qV1', 1, 'Ya#Ybbbbab', 145)
 ->('qV1', 0, 'Ya#Ybbbbab', 144)
 ->('IqR0', 1, 'Ya#Ybbbbab', 143)
 ->('qX0', 2, 'YX#Ybbbbab', 142)
 ->('qX1', 3, 'YX#Ybbbbab', 141)
 ->('qX1', 4, 'YX#Ybbbbab', 140)
 ->('qI0', 3, 'YX#Ybbbbab', 139)
 ->('qI0', 2, 'YX#bbbbbab', 138)
 ->('qI1', 3, 'YX#bbbbbab', 137)
 ->('qI2', 2, 'YX#Fbbbbab', 136)
 ->('qI3', 1, 'YX#Fbbbbab', 135)
 ->('qI3', 0, 'Ya#Fbbbbab', 134)
 ->('qI3', 7, '........ba#Fbbbbab', 133)
 ->('IqR0', 8, '........ba#Fbbbbab', 132)
 ->('qY0', 9, '........Ya#Fbbbbab', 131)
 ->('qY0', 10, '........Ya#Fbbbbab', 130)
 ->('qY1', 11, '........Ya#Fbbbbab', 129)
 ->('qY1', 12, '........Ya#Fbbbbab', 128)
 ->('qV0', 11, '........Ya#FYbbbab', 127)
 ->('qV0', 1

In [12]:
# Test 4
explore_tm(Asg6DTM, "bbb#bbabbba",150)

Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Accepted at  ('Fq0', 11, '........YYY#FFFYYYa', 24)
 via .. 
 ->('IqR0', 0, 'bbb#bbabbba', 150)
 ->('qY0', 1, 'Ybb#bbabbba', 149)
 ->('qY0', 2, 'Ybb#bbabbba', 148)
 ->('qY0', 3, 'Ybb#bbabbba', 147)
 ->('qY1', 4, 'Ybb#bbabbba', 146)
 ->('qV0', 3, 'Ybb#Ybabbba', 145)
 ->('qV1', 2, 'Ybb#Ybabbba', 144)
 ->('qV1', 1, 'Ybb#Ybabbba', 143)
 ->('qV1', 0, 'Ybb#Ybabbba', 142)
 ->('IqR0', 1, 'Ybb#Ybabbba', 141)
 ->('qY0', 2, 'YYb#Ybabbba', 140)
 ->('qY0', 3, 'YYb#Ybabbba', 139)
 ->('qY1', 4, 'YYb#Ybabbba', 138)
 ->('qY1', 5, 'YYb#Ybabbba', 137)
 ->('qV0', 4, 'YYb#YYabbba', 136)
 ->('qV0', 3, 'YYb#YYabbba', 135)
 ->('qV1', 2, 'YYb#YYabbba', 134)
 ->('qV1', 1, 'YYb#YYabbba', 133)
 ->('IqR0', 2, 'YYb#YYabbba', 132)
 ->('qY0', 3, 'YYY#YYabbba', 131)
 ->('qY1', 4, 'YYY#YYabbba', 130)
 ->('qY1', 5, 'YYY#YYabbba', 129)
 ->('qY1', 6, 'YYY#YYabbba', 128)
 ->('qI0', 5, 'YYY#YYabbba', 127)
 ->('qI0', 4, 'YYY#Ybabbba', 126)
 ->('qI0', 3,

In [13]:
# Test 5
explore_tm(Asg6DTM, "aa#bbaabbaa",50)

Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Accepted at  ('Fq0', 10, '........XX#FFXXbbaa', 3)
 via .. 
 ->('IqR0', 0, 'aa#bbaabbaa', 50)
 ->('qX0', 1, 'Xa#bbaabbaa', 49)
 ->('qX0', 2, 'Xa#bbaabbaa', 48)
 ->('qX1', 3, 'Xa#bbaabbaa', 47)
 ->('qI0', 2, 'Xa#bbaabbaa', 46)
 ->('qI1', 3, 'Xa#bbaabbaa', 45)
 ->('qI2', 2, 'Xa#Fbaabbaa', 44)
 ->('qI3', 1, 'Xa#Fbaabbaa', 43)
 ->('qI3', 0, 'Xa#Fbaabbaa', 42)
 ->('qI3', 7, '........aa#Fbaabbaa', 41)
 ->('IqR0', 8, '........aa#Fbaabbaa', 40)
 ->('qX0', 9, '........Xa#Fbaabbaa', 39)
 ->('qX0', 10, '........Xa#Fbaabbaa', 38)
 ->('qX1', 11, '........Xa#Fbaabbaa', 37)
 ->('qX1', 12, '........Xa#Fbaabbaa', 36)
 ->('qI0', 11, '........Xa#Fbaabbaa', 35)
 ->('qI0', 10, '........Xa#Fbaabbaa', 34)
 ->('qI1', 11, '........Xa#Fbaabbaa', 33)
 ->('qI1', 12, '........Xa#Fbaabbaa', 32)
 ->('qI2', 11, '........Xa#FFaabbaa', 31)
 ->('qI2', 10, '........Xa#FFaabbaa', 30)
 ->('qI3', 9, '........Xa#FFaabbaa', 29)
 ->('qI3', 8, '........Xa#

## Question 2 (50 points):


-  Design a non-deterministic Turing machine (NDTM) over the input alphabet Σ = {a, b, #} for the language of all strings of the form w#x where w, x ∈ {a, b}∗ and w is a substring of x. See the definition of substring here: https://en.wikipedia.org/wiki/Substring

- You must design a non-deterministic Turing machine—i.e., not design a DTM and call it an NDTM. You must employ a distinct phase of taking a guess (indicating it via your comments) and check the guess. A correct design will have a nondeterministic polynomial runtime, which means once the guess is made, only a polynomial zig-zag search must be involved to confirm the guess.

- More specifically, you will start the search at a random spot in x, and just do one comparison of w with "x beginning at that spot" and find that w is embedded within x beginning at that point. If it fails, you just block in a reject state. (Other non-deterministic options will search out other x-beginnings).

- You don't need to do any error-checks of user input. Assume that the user never gave anything outside of the above $\Sigma$ in their input.

- Add 5 tests on your own.

* The TM design fetches *25 points* for a well-documented TM

* The given tests passing fetches 3 points each - *total 15 points*

* Each test you add fetches 2 points each - *total 10 points*

* Strive to make three of your tests involve input strings of length 10 or more (to avoid having entirely short tests)

In [25]:
# w#x NDTM
Asg6NDTM = md2mc('''TM
!! Plan: Very similar to the DTM, except that we will not track failed substring starts.
!! Instead we will potentially just allow anything to be the start of a substring.

!! Reading step

IqR0 : #;#,R -> Fq0                        !! If we have read up to the midpoint guarantee an accept

IqR0 : X;X,R | Y;Y,R -> IqR0               !! Read past any X's or Y's at the start of the string.

IqR0 : a;X,R -> qX0                        !! Start the Validate X loop
IqR0 : b;Y,R -> qY0

!! Validate X loop

qX0 : a;a,R | b;b,R -> qX0                 !! Skip forward to the midpoint
qX0 : #;#,R -> qX1

qX1 : N;N,R | a;N,R | b;N,R -> qX1         !! Skip past any Ns. Also arbitrarily mark some a's and all b's as N to signify this may not be part of the substring.
qX1 : X;X,R | Y;Y,R -> qX2                 !! Skip any X's and Y's, and move to a state where we will no longer read any N's.
qX1 : a;X,L -> qS0                         !! Accept an a and return to the start of the string.

qX2 : X;X,R | Y;Y,R -> qX2                 !! The point of this state is just to ensure that once we've read an X we can't read any more Ns.
qX2 : a;X,L -> qS0                         !! This helps us clean up invalid branches more quickly.
                                           !! No transition on b's, because that just means it's invalid at this part of the substring.

!! Validate Y loop - very similar to X loop

qY0 : a;a,R | b;b,R -> qY0
qY0 : #;#,R -> qY1

qY1 : N;N,R | b;N,R | a;N,R -> qY1         !! Skip some amount of b's and all a's
qY1 : X;X,R | Y;Y,R -> qY2
qY1 : b;Y,L -> qS0                         !! Mark a b as a Y and return

qY2 : X;X,R | Y;Y,R -> qY2
qY2 : b;Y,L -> qS0

!! Return-to-start loop

qS0 : a;a,L | b;b,L -> qS0                 !! Just jump all the way back to the start
qS0 : X;X,L | Y;Y,L -> qS0
qS0 : N;N,L | #;#,L -> qS0

qS0 : .;.,R -> IqR0

''')


### Test Cases: The "fuel" argument of 50 may be increased as needed (start with 50 and that is good if all tests finish)

In [26]:
explore_tm(Asg6NDTM, "aba#aba",50)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('Fq0', 12, '........XYX#XYX', 10)
 via .. 
 ->('IqR0', 0, 'aba#aba', 50)
 ->('qX0', 1, 'Xba#aba', 49)
 ->('qX0', 2, 'Xba#aba', 48)
 ->('qX0', 3, 'Xba#aba', 47)
 ->('qX1', 4, 'Xba#aba', 46)
 ->('qS0', 3, 'Xba#Xba', 45)
 ->('qS0', 2, 'Xba#Xba', 44)
 ->('qS0', 1, 'Xba#Xba', 43)
 ->('qS0', 0, 'Xba#Xba', 42)
 ->('qS0', 7, '........Xba#Xba', 41)
 ->('IqR0', 8, '........Xba#Xba', 40)
 ->('IqR0', 9, '........Xba#Xba', 39)
 ->('qY0', 10, '........XYa#Xba', 38)
 ->('qY0', 11, '........XYa#Xba', 37)
 ->('qY1', 12, '........XYa#Xba', 36)
 ->('qY2', 13, '........XYa#Xba', 35)
 ->('qS0', 12, '........XYa#XYa', 34)
 ->('qS0', 11, '........XYa#XYa', 33)
 ->('qS0', 10, '........XYa#XYa', 32)
 ->('qS0', 9, '........XYa#XYa', 31)
 ->('qS0', 8, '........XYa#XYa', 30)
 ->('qS0', 7, '........XYa#XYa', 29)

In [27]:
explore_tm(Asg6NDTM, "aba#ab",50)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('qX2', 14, '........XYX#XY', 22)
 via .. 
 ->('IqR0', 0, 'aba#ab', 50)
 ->('qX0', 1, 'Xba#ab', 49)
 ->('qX0', 2, 'Xba#ab', 48)
 ->('qX0', 3, 'Xba#ab', 47)
 ->('qX1', 4, 'Xba#ab', 46)
 ->('qS0', 3, 'Xba#Xb', 45)
 ->('qS0', 2, 'Xba#Xb', 44)
 ->('qS0', 1, 'Xba#Xb', 43)
 ->('qS0', 0, 'Xba#Xb', 42)
 ->('qS0', 7, '........Xba#Xb', 41)
 ->('IqR0', 8, '........Xba#Xb', 40)
 ->('IqR0', 9, '........Xba#Xb', 39)
 ->('qY0', 10, '........XYa#Xb', 38)
 ->('qY0', 11, '........XYa#Xb', 37)
 ->('qY1', 12, '........XYa#Xb', 36)
 ->('qY2', 13, '........XYa#Xb', 35)
 ->('qS0', 12, '........XYa#XY', 34)
 ->('qS0', 11, '........XYa#XY', 33)
 ->('qS0', 10, '........XYa#XY', 32)
 ->('qS0', 9, '........XYa#XY', 31)
 ->('qS0', 8, '........XYa#XY', 30)
 ->('qS0', 7, '........XYa#XY', 29)
 ->('IqR0', 8, '........XYa#XY', 28)
 ->('IqR0', 9, '........

In [28]:
explore_tm(Asg6NDTM, "aba#baba",50)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('Fq0', 12, '........XYX#NXYX', 4)
 via .. 
 ->('IqR0', 0, 'aba#baba', 50)
 ->('qX0', 1, 'Xba#baba', 49)
 ->('qX0', 2, 'Xba#baba', 48)
 ->('qX0', 3, 'Xba#baba', 47)
 ->('qX1', 4, 'Xba#baba', 46)
 ->('qX1', 5, 'Xba#Naba', 45)
 ->('qS0', 4, 'Xba#NXba', 44)
 ->('qS0', 3, 'Xba#NXba', 43)
 ->('qS0', 2, 'Xba#NXba', 42)
 ->('qS0', 1, 'Xba#NXba', 41)
 ->('qS0', 0, 'Xba#NXba', 40)
 ->('qS0', 7, '........Xba#NXba', 39)
 ->('IqR0', 8, '........Xba#NXba', 38)
 ->('IqR0', 9, '........Xba#NXba', 37)
 ->('qY0', 10, '........XYa#NXba', 36)
 ->('qY0', 11, '........XYa#NXba', 35)
 ->('qY1', 12, '........XYa#NXba', 34)
 ->('qY1', 13, '........XYa#NXba', 33)
 ->('qY2', 14, '........XYa#NXba', 32)
 ->('qS0', 13, '........XYa#NXYa', 31)
 ->('qS0', 12, '........XYa#NXYa', 30)
 ->('qS0', 11, '........XYa#NXY

In [29]:
explore_tm(Asg6NDTM, "aba#babba",50)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('qX2', 15, '........XYX#NXYba', 17)
 via .. 
 ->('IqR0', 0, 'aba#babba', 50)
 ->('qX0', 1, 'Xba#babba', 49)
 ->('qX0', 2, 'Xba#babba', 48)
 ->('qX0', 3, 'Xba#babba', 47)
 ->('qX1', 4, 'Xba#babba', 46)
 ->('qX1', 5, 'Xba#Nabba', 45)
 ->('qS0', 4, 'Xba#NXbba', 44)
 ->('qS0', 3, 'Xba#NXbba', 43)
 ->('qS0', 2, 'Xba#NXbba', 42)
 ->('qS0', 1, 'Xba#NXbba', 41)
 ->('qS0', 0, 'Xba#NXbba', 40)
 ->('qS0', 7, '........Xba#NXbba', 39)
 ->('IqR0', 8, '........Xba#NXbba', 38)
 ->('IqR0', 9, '........Xba#NXbba', 37)
 ->('qY0', 10, '........XYa#NXbba', 36)
 ->('qY0', 11, '........XYa#NXbba', 35)
 ->('qY1', 12, '........XYa#NXbba', 34)
 ->('qY1', 13, '........XYa#NXbba', 33)
 ->('qY2', 14, '........XYa#NXbba', 32)
 ->('qS0', 13, '........XYa#NXYba', 31)
 ->('qS0', 12, '........XYa#NXYba', 30)
 ->('qS0

In [31]:
explore_tm(Asg6NDTM, "bba#bbabba",100)

Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Rejected at  ('qY1', 10, 'Yba#NNNNNN', 90)
 via .. 
 ->('IqR0', 0, 'bba#bbabba', 100)
 ->('qY0', 1, 'Yba#bbabba', 99)
 ->('qY0', 2, 'Yba#bbabba', 98)
 ->('qY0', 3, 'Yba#bbabba', 97)
 ->('qY1', 4, 'Yba#bbabba', 96)
 ->('qY1', 5, 'Yba#Nbabba', 95)
 ->('qY1', 6, 'Yba#NNabba', 94)
 ->('qY1', 7, 'Yba#NNNbba', 93)
 ->('qY1', 8, 'Yba#NNNNba', 92)
 ->('qY1', 9, 'Yba#NNNNNa', 91)
 ->('qY1', 10, 'Yba#NNNNNN', 90)
Rejected at  ('qY2', 17, '........YYa#NNNNYa', 73)
 via .. 
 ->('IqR0', 0, 'bba#bbabba', 100)
 ->('qY0', 1, 'Yba#bbabba', 99)
 ->('qY0', 2, 'Yba#bbabba', 98)
 ->('qY0', 3, 'Yba#bbabba', 97)
 ->('qY1', 4, 'Yba#bbabba', 96)
 ->('qY1', 5, 'Yba#Nbabba', 95)
 ->('qY1', 6, 'Yba#NNabba', 94)
 ->('qY1', 7, 'Yba#NNNbba', 93)
 ->('qY1', 8, 'Yba#NNNNba', 92)
 ->('qS0', 7,

### 5 Test Cases You Add

In [35]:
# Test 1
explore_tm(Asg6NDTM, "abb#ababb",100)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Rejected at  ('qY2', 14, '........XYY#XYabb', 72)
 via .. 
 ->('IqR0', 0, 'abb#ababb', 100)
 ->('qX0', 1, 'Xbb#ababb', 99)
 ->('qX0', 2, 'Xbb#ababb', 98)
 ->('qX0', 3, 'Xbb#ababb', 97)
 ->('qX1', 4, 'Xbb#ababb', 96)
 ->('qS0', 3, 'Xbb#Xbabb', 95)
 ->('qS0', 2, 'Xbb#Xbabb', 94)
 ->('qS0', 1, 'Xbb#Xbabb', 93)
 ->('qS0', 0, 'Xbb#Xbabb', 92)
 ->('qS0', 7, '........Xbb#Xbabb', 91)
 ->('IqR0', 8, '........Xbb#Xbabb', 90)
 ->('IqR0', 9, '........Xbb#Xbabb', 89)
 ->('qY0', 10, '........XYb#Xbabb', 88)
 ->('qY0', 11, '........XYb#Xbabb', 87)
 ->('qY1', 12, '........XYb#Xbabb', 86)
 ->('qY2', 13, '........XYb#Xbabb', 85)
 ->('qS0', 12, '........XYb#XYabb', 84)
 ->('qS0', 11, '........XYb#XYabb', 83)
 ->('qS0', 10, '........XYb#XYabb', 82)
 ->('qS0', 9, '........XYb#XYabb', 81)
 ->('qS0', 8, '........XYb#XYabb', 80)
 ->('qS0', 7, '........XYb#XYab

In [37]:
# Test 2
explore_tm(Asg6NDTM, "ab#bbbbbab",150)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('Fq0', 11, '........XY#NNNNNXY', 109)
 via .. 
 ->('IqR0', 0, 'ab#bbbbbab', 150)
 ->('qX0', 1, 'Xb#bbbbbab', 149)
 ->('qX0', 2, 'Xb#bbbbbab', 148)
 ->('qX1', 3, 'Xb#bbbbbab', 147)
 ->('qX1', 4, 'Xb#Nbbbbab', 146)
 ->('qX1', 5, 'Xb#NNbbbab', 145)
 ->('qX1', 6, 'Xb#NNNbbab', 144)
 ->('qX1', 7, 'Xb#NNNNbab', 143)
 ->('qX1', 8, 'Xb#NNNNNab', 142)
 ->('qS0', 7, 'Xb#NNNNNXb', 141)
 ->('qS0', 6, 'Xb#NNNNNXb', 140)
 ->('qS0', 5, 'Xb#NNNNNXb', 139)
 ->('qS0', 4, 'Xb#NNNNNXb', 138)
 ->('qS0', 3, 'Xb#NNNNNXb', 137)
 ->('qS0', 2, 'Xb#NNNNNXb', 136)
 ->('qS0', 1, 'Xb#NNNNNXb', 135)
 ->('qS0', 0, 'Xb#NNNNNXb', 134)
 ->('qS0', 7, '........Xb#NNNNNXb', 133)
 ->('IqR0', 8, '........Xb#NNNNNXb', 132)
 ->('IqR0', 9, '........Xb#NNNNNXb', 131)
 ->('qY0', 10, '........XY#NNNNNXb', 130)
 ->('qY1', 11, '........XY#NNNNNXb', 129)
 ->('qY1', 12, '........XY#NNNNNXb', 128)
 ->('qY1', 13

In [38]:
# Test 3
explore_tm(Asg6NDTM, "ba#bbbbbab",50)

Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Rejected at  ('qY1', 10, 'Ya#NNNNNNN', 40)
 via .. 
 ->('IqR0', 0, 'ba#bbbbbab', 50)
 ->('qY0', 1, 'Ya#bbbbbab', 49)
 ->('qY0', 2, 'Ya#bbbbbab', 48)
 ->('qY1', 3, 'Ya#bbbbbab', 47)
 ->('qY1', 4, 'Ya#Nbbbbab', 46)
 ->('qY1', 5, 'Ya#NNbbbab', 45)
 ->('qY1', 6, 'Ya#NNNbbab', 44)
 ->('qY1', 7, 'Ya#NNNNbab', 43)
 ->('qY1', 8, 'Ya#NNNNNab', 42)
 ->('qY1', 9, 'Ya#NNNNNNb', 41)
 ->('qY1', 10, 'Ya#NNNNNNN', 40)
Rejected at  ('qX2', 18, '........YX#NNNNNNY', 20)
 via .. 
 ->('IqR0', 0, 'ba#bbbbbab', 50)
 ->('qY0', 1, 'Ya#bbbbbab', 49)
 ->('qY0', 2, 'Ya#bbbbbab', 48)
 ->('qY1', 3, 'Ya#bbbbbab', 47)
 ->('qY1', 4, 'Ya#Nbbbbab', 46)
 ->('qY1', 5, 'Ya#NNbbbab

In [40]:
# Test 4
explore_tm(Asg6NDTM, "bbb#bbabbba",100)

Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Detailing the halted configs now.
Rejected at  ('qY1', 11, 'Ybb#NNNNNNN', 89)
 via .. 
 ->('IqR0', 0, 'bbb#bbabbba', 100)
 ->('qY0', 1, 'Ybb#bbabbba', 99)
 ->('qY0', 2, 'Ybb#bbabbba', 98)
 ->('qY0', 3, 'Ybb#bbabbba', 97)
 ->('qY1', 4, 'Ybb#bbabbba', 96)
 ->('qY1', 5, 'Ybb#Nbabbba', 95)
 ->('qY1', 6, 'Ybb#NNabbba', 94)
 ->('qY1', 7, 'Ybb#NNNbbba', 93)
 ->('qY1', 8, 'Ybb#NNNNbba', 92)
 ->('qY1', 9, 'Ybb#NNNNNba', 91)
 ->('qY1', 10, 'Ybb#NNNNNNa', 90)
 ->('qY1', 11, 'Ybb#NNNNNNN', 89)
Rejected at  ('qY2', 18, '........YYb#NNNNNYa', 70)
 via .. 
 ->('IqR0', 0, 'bbb#bbabbba', 100)
 ->('qY0', 1, 'Ybb#bbabbba', 99)
 ->('qY0', 2, 'Ybb#bbabbba', 98)
 ->('qY0', 3, 'Ybb#bbabbba', 97)
 ->('qY1', 4, 'Ybb#bbabbba', 96)
 ->('qY1', 5, 'Ybb#Nbabbba', 95)
 ->('qY1', 6, 'Yb

In [42]:
# Test 5
explore_tm(Asg6NDTM, "aa#bbaabbaa",50)

Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the LEFT!
Allocating  8  tape cells to the RIGHT!
Allocating  8  tape cells to the RIGHT!
Detailing the halted configs now.
Accepted at  ('Fq0', 11, '........XX#NNXXbbaa', 21)
 via .. 
 ->('IqR0', 0, 'aa#bbaabbaa', 50)
 ->('qX0', 1, 'Xa#bbaabbaa', 49)
 ->('qX0', 2, 'Xa#bbaabbaa', 48)
 ->('qX1', 3, 'Xa#bbaabbaa', 47)
 ->('qX1', 4, 'Xa#Nbaabbaa', 46)
 ->('qX1', 5, 'Xa#NNaabbaa', 45)
 ->('qS0', 4, 'Xa#NNXabbaa', 44)
 ->('qS0', 3, 'Xa#NNXabbaa', 43)
 ->('qS0', 2, 'Xa#NNXabbaa', 42)
 ->('qS0', 1, 'Xa#NNXabbaa', 41)
 ->('qS0', 0, 'Xa#NNXabbaa', 40)
 ->('qS0', 7, '........Xa#NNXabbaa', 39)
 ->('IqR0', 8, '........Xa#NNXabbaa', 38)
 ->('IqR0', 9, '........Xa#NNXabbaa', 37)
 ->('qX0', 10, '........XX#NNXabbaa', 36)
 ->('qX1', 11, '........XX#NNXabbaa', 35)
 ->('qX1', 12, '........XX#NNXabbaa', 34)
 ->('qX1', 13, '........XX#NNXabbaa', 33)
 ->('qX2', 1

# END
